# Interaction with the World Homework (#3)
Python Computing for Data Science (c) J Bloom, UC Berkeley, 2016

# 1) Monty: The Python Siri

Let's make a Siri-like program with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

In [24]:
import pyaudio
import wave

chunk = 2048
FORMAT = pyaudio.paInt32
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "new2.wav"
p = pyaudio.PyAudio()
stream = p.open(format = FORMAT,
    channels = CHANNELS,
    rate = RATE,
    input = True,
    frames_per_buffer = chunk)
all = []
for i in range(0, int(RATE / chunk * RECORD_SECONDS)):
    data = stream.read(chunk)
    all.append(data)
print("* done recording")
stream.close()

p.terminate()


# waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
# waveFile.setnchannels(CHANNELS)
# waveFile.setsampwidth(p.get_sample_size(FORMAT))
# # waveFile.setsampwidth(2)
# waveFile.setframerate(RATE)
# waveFile.writeframes(b''.join(all))
# waveFile.close()


from googlevoice import Voice
from googlevoice.util import input

voice = Voice()
voice.login()

phoneNumber = input('Number to send message to: ')
text = input('Message text: ')

voice.send_sms(phoneNumber, text)

# import houndify
# import os
# import time

# os.environ['HOUNDIFY_API_CLIENT']='QfZC7UHC__Se1iiCpJdGbg=='
# os.environ['HOUNDIFY_API_KEY']='3uCW49-HHTKw4xt2MizSe5XfIa3TrkePApdYMWBtBfWN6bC79ISWPzdwRnZ-NQwu3eyswc_M8SPjodehm20aFQ=='

# CLIENT_ID = os.environ.get('HOUNDIFY_API_CLIENT')
# CLIENT_KEY = os.environ.get('HOUNDIFY_API_KEY')
# BUFFER_SIZE = 512

# #
# # Simplest HoundListener; just print out what we receive.
# # You can use these callbacks to interact with your UI.
# #
# class MyListener(houndify.HoundListener):
#   def onPartialTranscript(self, transcript):
#     print("Partial transcript: " + transcript)
#   def onFinalResponse(self, response):
#     print("Final response: " + str(response))
#   def onError(self, err):
#     print("Error: " + str(err))


# client = houndify.StreamingHoundClient(CLIENT_ID, CLIENT_KEY, "test_user")
# print(client)
# audio = wave.open(WAVE_OUTPUT_FILENAME)
# if audio.getsampwidth() != 2:
#   print("%s: wrong sample width (must be 16-bit)" % WAVE_OUTPUT_FILENAME)
# if audio.getframerate() != 8000 and audio.getframerate() != 16000:
#   print("%s: unsupported sampling frequency (must be either 8 or 16 khz)" % WAVE_OUTPUT_FILENAME)
# if audio.getnchannels() != 1:
#   print("%s: must be single channel (mono)" % WAVE_OUTPUT_FILENAME)

# client.setSampleRate(audio.getframerate())
# client.start(MyListener())

# while True:
#   samples = audio.readframes(BUFFER_SIZE)
#   if len(samples) == 0: break
#   if client.fill(samples): break
#   time.sleep(0.032) # simulate real-time so we can see the partial transcripts
       
# result = client.finish() # returns either final response or error



* done recording


TypeError: get() got an unexpected keyword argument 'raw'

# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/fall2016/ay250/resources -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.